In [1]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

In [2]:
N = 400
feats = 784
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

In [3]:
# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats), name="w")
b = theano.shared(0., name="b")
print("Initial model:")
print(w.get_value())
print(b.get_value())

Initial model:
[ -1.30348546e+00  -2.21067363e+00  -6.65608937e-02  -2.13349888e-01
  -3.82627450e-01   1.23164120e-01  -3.09917174e-01  -1.70586394e+00
   4.35086141e-01  -5.63291531e-01  -1.23873196e+00   2.10854372e+00
   5.20877494e-02  -7.69288886e-01  -7.36984671e-01  -3.15278034e-01
  -1.16975295e+00  -1.30541078e+00   1.42999148e+00  -1.51331810e+00
  -2.41613207e-01   2.99407375e-01   2.99077155e-01  -4.04967898e-01
   3.23334725e-01   1.22272425e+00  -1.87254574e-01  -2.29181559e-01
  -2.51526715e+00   8.65733404e-02  -1.58999093e+00   6.83718817e-01
  -1.04379745e+00  -1.72200790e-01  -1.71097637e+00   2.72467786e-01
  -6.13851938e-01  -3.76648685e-01   1.29857734e+00   4.94752900e-01
  -1.28333587e+00   2.26351035e+00  -5.28437699e-01  -1.23663002e-01
  -5.93616387e-02  -6.27662926e-01   9.60085336e-01   5.09248481e-01
  -2.03213696e+00  -7.41107279e-01  -4.10755382e-01   8.44052570e-01
   5.26865418e-01  -5.35725266e-01   1.49068848e+00  -1.49891547e+00
   3.76393590e-01  

In [4]:
# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # (we shall return to this in a
                                          # following section of this tutorial)

In [5]:
# Compile
train = theano.function(inputs = [x,y],
                        outputs = [prediction, xent],
                        updates = ((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs = prediction)

In [6]:
# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
[ -7.19961209e-02  -1.54529100e-01   8.85333260e-02   1.54052002e-01
  -5.34341636e-02  -1.32020922e-02  -3.04844484e-02  -4.00060569e-02
   9.49840454e-02   2.94855021e-02  -2.19437745e-01  -1.27247863e-01
  -5.68292007e-03   7.08693331e-02  -1.41432096e-01   6.15046968e-02
  -1.04788293e-01  -1.76824958e-01   1.01292077e-01  -9.52279553e-02
  -6.65484838e-02   7.85339652e-03  -4.21859893e-02   4.99423757e-02
   8.01061549e-02   1.57070843e-02  -5.64720323e-02  -1.25112656e-03
   3.18734410e-03  -3.67635922e-02  -9.55293842e-05   1.00278003e-01
  -3.07085593e-02   1.72579672e-01  -1.67291644e-01   2.40286646e-01
  -1.23155874e-02   6.49845982e-02   1.73571141e-01  -2.15621184e-02
   8.84407142e-02   6.36183818e-02   4.25095605e-02  -2.02204057e-01
   8.65377580e-02  -8.11913418e-03   9.62665804e-03   1.52726502e-02
   2.22317927e-02  -1.57678985e-01   7.50791111e-02   1.11356992e-01
   1.23392884e-01   5.84267089e-02   5.65496128e-02  -4.35136662e-02
   1.20542739e-01   5